In [ ]:
#!pip install import_ipynb --user
#!pip install ipynb --user

In [1]:
from ipynb.fs.full.Notebook_for_Noronet_pie import *
#from Notebook_for_Noronet_pie import *

In [2]:
import pandas as pd
from datetime import datetime as dt
import json, time, os
from bokeh.resources import CDN
from bokeh.embed import file_html, json_item
from bokeh.plotting import figure
from bokeh.io import curdoc, output_notebook, show 
from bokeh.document import Document
from bokeh.resources import INLINE
from bokeh.util.browser import view


# import holoviews as hv
# hv.extension('bokeh')
# from holoviews import opts


from bokeh.models import DatetimeTickFormatter, Legend, NumeralTickFormatter
from bokeh.layouts import widgetbox, row, layout
from bokeh.models import ColumnDataSource, Toggle, BoxAnnotation, CustomJS
from bokeh.models.widgets import DataTable, DateFormatter, TableColumn
from bokeh.models import ColumnDataSource, Plot, LinearAxis, Grid, Circle, HoverTool, BoxSelectTool,\
    SaveTool, CustomJS, DatetimeAxis, LinearAxis, NumeralTickFormatter, FactorRange, CategoricalColorMapper,\
    Legend
from bokeh.models.widgets.tables import (
    DataTable, TableColumn, IntEditor
)
from bokeh.models.widgets import DataTable, TableColumn, StringFormatter,\
        NumberFormatter, StringEditor, IntEditor, NumberEditor, SelectEditor,\
        CheckboxButtonGroup, CheckboxGroup, MultiSelect, RadioButtonGroup,\
        Select, Slider, Panel, Tabs, TextInput, Paragraph, Div, Button, Dropdown,\
        DateRangeSlider


from bokeh.palettes import Magma9, Category20b, Category20c
from bokeh.transform import factor_cmap

from bokeh.models.glyphs import Circle
from bokeh.models.layouts import Column, Row
from bokeh.layouts import gridplot

In [14]:
import logging
logging.basicConfig(filename='regiapp.log', filemode='w', format='%(asctime)s - %(name)s - %(levelname)s - %(message)s', level=logging.DEBUG)


### Simple pie chart for all data

#### Figure 5. GI polymerase genotypes per country (>10 submitted sequences)

In [3]:
output_notebook()

Loading BokehJS ...

In [4]:
# def on_event_slider(attr, old, new):
#     start = dt.utcfromtimestamp(new[0]/1000)
#     end = dt.utcfromtimestamp(new[1]/1000)
#     e = curdoc().get_model_by_name('pie')
#     e.children=[create_country_pie(gi_df[gi_df['Sample Date'].apply(lambda x: True if ((x > start) and (x < end)) else False)], g_type="gii")]
#     time.sleep(.2)
    
#date_range_slider.on_change('value', on_event_slider)Message posted 21.4.20.

In [5]:
# #choices = list(df.Country.unique())
# choices = [ 'GI-%s'%s for s in list(gi_df['ORF1_sub'].unique())]
# choices.extend([ 'GII-%s'%s for s in list(gii_df['ORF1_sub'].unique())])
# choices.sort()
# checkbox_group1 = CheckboxGroup(labels=[t for t in choices if 'GI-' in t], active=[], inline=True)
# checkbox_group2 = CheckboxGroup(labels=[t for t in choices if 'GII-' in t], active=[], inline=True)
# d = Div(text="""Choose type""")
# div_choose = Column(d, Row(Column(checkbox_group1, checkbox_group2)))


# Lay = Column(div_choose, tags=['choice'])
# Lay.children[0].children.append(b_choose)
# show(Lay)

In [6]:
# html={}
# html['target_id'] = "myplot2"
# html['root_id'] = 3245
# html['doc'] = doc.to_json()

In [7]:
          
# df, dict_categories = load_data('noronet_clean.fr', "categories.json")
# # checkbox
# c_choices = list(df.Country.unique())
# c_choices.sort()
# checkbox_group = CheckboxGroup(labels=c_choices, active=[], inline=True)

# # divs
# d = Div(text="""Choose country""")
# div_choose = Column(d, Row(Column(checkbox_group)))

# # button to replot
# b_choose = Button(label="Plot", button_type="success")#, callback=CustomJS.from_py_func(on_change_save))
# b_choose.on_click(on_click_plot)

# # time slider
# dates = dict_categories['dates']
# min_date = df[dates[0]].min()
# max_date = df[dates[0]].max()
# date_range_slider = DateRangeSlider(title="Date Range: ", start=min_date, end=max_date, value=(min_date, max_date), step=1, name="dateslider")

# # pie charts
# pie_plot = create_country_pie(df, countries=[], genotype="gi", orf_type="ORF1_sub")

# # put together layout
# Lay = Column(div_choose, Row(pie_plot, tags=['plott'], name="pie"), tags=['choice'])

# df, dict_categories = load_data('noronet_clean.fr', "categories.json")
# #curdoc().add_root(Column(d, date_range_slider, b_choose, Lay))

# create_url_text(curdoc(), slider=['dateslider'])

In [8]:

def get_slider_state(slider):
    start_date = slider.value[0]
    end_date = slider.value[1]
    if isinstance(start_date, int):
        start_date = dt.utcfromtimestamp(slider.value[0]/1000).strftime("%d-%m-%Y")
        end_date = dt.utcfromtimestamp(slider.value[1]/1000).strftime("%d-%m-%Y")
        start_date = pd.to_datetime(start_date)
        end_date = pd.to_datetime(end_date)
    else:
        start_date = slider.value[0]#.strftime("%d-%m-%Y")
        end_date = slider.value[1]#.strftime("%d-%m-%Y")
        
    return dict(start_date=start_date, end_date=end_date, type='slider', name=slider.name)

def get_checkbox_state(checkbox):
    values = [checkbox.labels[a] for a in checkbox.active]
    return dict(values=values, type='checkbox', name='checkbox')

def create_url_text(layout, **kwargs):
    field_data = {}
    for key in kwargs.keys():
        for item_name in kwargs[key]:
            if key == 'slider':
                item = layout.get_model_by_name(item_name)
                field_data[item_name] =  get_slider_state(item)
            if key == 'checkbox':
                item = layout.get_model_by_name(item_name)
                field_data[item_name] =  get_checkbox_state(item)
                
    # Create fields
    fields = "?"
    for keys in field_data:
        item = field_data[keys]
        if item['type'] == 'slider':
            fields +="&{name}_values={start_date},{end_date}".format(**item)
        if item['type'] == 'checkbox':
            if item['values']:
                fields +="&{0}_values={1}".format(item['name'], ",".join(item['values']))
            
    # + "&" + "date=%s,%s"%(start_date, end_date) 
    api_resource = os.getenv("NB_URL")
    #api_host = os.getenv("NB_HOST")
    api_host = "https://kooplex-fiek.elte.hu"
    url = os.path.join(api_host, api_resource, "report", fields)
    return url



In [9]:
def load_data(f_dataframe, f_categories):
    # dataframe
    df = pd.read_feather(f_dataframe)

    # categories
    dict_categories = {}

    with open(f_categories, 'r') as f:
        dict_categories = json.loads(f.read())

    # del unkxnown
    df = df[df['ORF1_sub']!='Unknown']

    #df['Sample Date'] = df['Sample Date'].astype(np.int)

    #gi_df = df[df['Genus-Genogroup']=='Caliciviridae Norovirus GI']
    #gii_df = df[df['Genus-Genogroup']=='Caliciviridae Norovirus GII']
    return df, dict_categories

def on_click_plot(new):   

    slider = curdoc().gterminalset_model_by_name('dateslider')
    newtime = get_slider_state(slider)
    start, end = newtime['start_date'], newtime['end_date']
    logging.debug('start: %s, end: %s'%(start, end))

    checkbox = curdoc().get_model_by_name('country_checkbox')
    countries = get_checkbox_state(checkbox)['values']    
    e = curdoc().get_model_by_name('pie_div')
    tdf = df[df['Sample Date'].apply(lambda x: True if ((x > start) and (x < end)) else False)]
    pie_plot = create_country_pie(tdf, countries=countries, genotype="gi", orf_type="ORF1_sub")
    e.children=[pie_plot]
    endpoint = curdoc().get_model_by_name('endpoint_pie')
    endpoint.text = create_url_text(curdoc(), slider=['dateslider'], checkbox=['country_checkbox'])

    
    
#curdoc().add_root(Column(d, date_range_slider,b_choose))

In [10]:
df, dict_categories = load_data('noronet_clean.fr', "categories.json")

In [11]:
# slider = curdoc().get_model_by_name('dateslider')
# newtime = get_slider_state(slider)
# start, end = newtime['start_date'], newtime['end_date']

# df[df['Sample Date'].apply(lambda x: True if ((x > start) and (x < end)) else False)]

In [13]:
def complete_layout(df, dict_categories):
    # checkbox
    c_choices = list(df.Country.unique())
    c_choices.sort()
    checkbox_group = CheckboxGroup(labels=c_choices, active=[], inline=True, name='country_checkbox')

    # divs
    d = Div(text="""Choose country""")
    div_choose = Column(d, Row(Column(checkbox_group)))

    # button to replot
    b_choose = Button(label="Plot", button_type="success")#, callback=CustomJS.from_py_func(on_change_save))
    b_choose.on_click(on_click_plot)

    # time slider
    dates = dict_categories['dates']
    min_date = df[dates[0]].min()
    max_date = df[dates[0]].max()
    date_range_slider = DateRangeSlider(title="Date Range: ", start=min_date, end=max_date, value=(min_date, max_date), step=1, name='dateslider')


    # pie charts
    pie_plot = create_country_pie(df, countries=[], genotype="gi", orf_type="ORF1_sub")
    item_to_be_treplaced = Row(pie_plot, tags=['plott'], name="pie_div")

    # put together layout
    Lay = Column(d, date_range_slider, b_choose, div_choose, item_to_be_treplaced, name='main')
    c = curdoc()
    c.add_root(Lay)

    # api url
    api_url = create_url_text(c, slider=['dateslider'], checkbox=['country_checkbox'])
    api_url_div = Div(text="<h3>%s</h3>"%api_url, name='endpoint_pie')
    c.remove_root(c.roots[0])


    return Column( Lay, api_url_div)

### Hierarchic pie chart
https://stackoverflow.com/questions/33019879/hierarchic-pie-donut-chart-from-pandas-dataframe-using-bokeh-or-matplotlib

<img src="http://i.stack.imgur.com/YKbzU.png" width="300" />

### Figure 8 Outbreak settings for GI, GII.2, GII.4, GII.6 and GII.17.

## How to create easily
* choose an ORF1_sub type

### Multiple charts
* Use Multichoice or multiselect
* generate all

In [ ]:
# import io
# import random

# from jinja2 import Template

# from bokeh.embed import components
# from bokeh.plotting import figure
# from bokeh.resources import INLINE
# from bokeh.util.browser import view

# ########## BUILD FIGURES ################

# PLOT_OPTIONS = dict(plot_width=800, plot_height=300)
# SCATTER_OPTIONS = dict(size=12, alpha=0.5)

# data = lambda: [random.choice([i for i in range(100)]) for r in range(10)]

# red = figure(sizing_mode='scale_width', tools='pan', **PLOT_OPTIONS)
# red.scatter(data(), data(), color="red", **SCATTER_OPTIONS)

# blue = figure(sizing_mode='fixed', tools='pan', **PLOT_OPTIONS)
# blue.scatter(data(), data(), color="blue", **SCATTER_OPTIONS)

# green = figure(sizing_mode='scale_width', tools='pan', **PLOT_OPTIONS)
# green.scatter(data(), data(), color="green", **SCATTER_OPTIONS)

# ########## RENDER PLOTS ################

# # Define our html template for out plots
# template = Template("""\
# <!DOCTYPE html>
# <html lang="en">
#     <head>
#         <meta charset="utf-8">
#         <title>Responsive plots</title>
#         {{ resources }}
#     </head>
#     <body>
#     <h2>Resize the window to see some plots resizing</h2>
#     <h3>Red - pan tool, scale_width</h3>
#     {{ plot_div.red }}
#     <h3>Green - pan tool, scale_width</h3>
#     {{ plot_div.green }}
#     <h3>Blue - pan tool, fixed/h3>
#     {{ plot_div.blue }}

#     {{ plot_script }}
#     </body>
# </html>
# """)

# resources = INLINE.render()

# script, div = components({'red': red, 'blue': blue, 'green': green})

# html = template.render(resources=resources,
#                        plot_script=script,
#                        plot_div=div)

# filename = 'embed_multiple_responsive.html'

# with io.open(filename, mode='w', encoding='utf-8') as f:
#     f.write(html)

# view(filename)
